In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plt.style.use("ggplot")

In [3]:
competition = 'asl-signs'

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('..') / 'input'  / 'competition'
    # !pip install -Uqq fastai
else:
    import zipfile, kaggle
    path = Path.home() / '.data' / 'asl-signs'
    if not path.exists():
        path.mkdir(exist_ok=True)
        kaggle.api.competition_download_cli(competition, path=path)
        zipfile.ZipFile(path / f'{competition}.zip').extractall(path)

# Labeled Data

In [4]:
with open(path / 'sign_to_prediction_index_map.json') as f:
    sign_labels = json.load(f)

In [5]:
train = (pd.read_csv(path / 'train_with_meta.csv')
         .assign(idx=lambda x: x.participant_id.astype(str) + '_' + x.sequence_id.astype(str))
         .set_index('idx')
        )
train.head()

,path,participant_id,sequence_id,sign,cnt_partial_nulls,cnt_partial_nulls_by_frame,total_frames,face,left_hand,pose,right_hand
idx,,,,,,,,,,,
26734_1000035562,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,0.0,0.0,23.0,23.0,0.0,23.0,11.0
28656_1000106739,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,0.0,0.0,11.0,11.0,0.0,11.0,2.0
16069_100015657,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,0.0,0.0,105.0,105.0,28.0,105.0,0.0
25571_1000210073,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,0.0,0.0,12.0,12.0,0.0,12.0,12.0
62590_1000240708,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,0.0,0.0,18.0,18.0,0.0,18.0,18.0


In [6]:
"""
rounding_n = 2
(landmarks.groupby('type')
 .agg(
    cnt_landmarks=pd.NamedAgg('landmark_index', 'nunique'),
    cnt_frames=pd.NamedAgg('frame', 'nunique'),
    x_min=pd.NamedAgg('x', 'min'),
    x_max=pd.NamedAgg('x', 'max'),
    y_min=pd.NamedAgg('y', 'min'),
    y_max=pd.NamedAgg('y', 'max'),
    z_min=pd.NamedAgg('z', 'min'),
    z_max=pd.NamedAgg('z', 'max'),
 )
 .assign(
     x_min=lambda x: x.x_min.round(rounding_n),
     x_max=lambda x: x.x_max.round(rounding_n),
     y_min=lambda x: x.y_min.round(rounding_n),
     y_max=lambda x: x.y_max.round(rounding_n),
     z_min=lambda x: x.z_min.round(rounding_n),
     z_max=lambda x: x.z_max.round(rounding_n),
 )
 .T
)
"""
pass

In [7]:
def interpolate_values(landmarks):
    idx = pd.MultiIndex.from_product(
        [frames, landmarks.landmark_index.unique().tolist()],
        names=['frame', 'landmark_index']
    )

    landmarks = (landmarks.set_index(['frame', 'landmark_index'])
                 .reindex(idx, fill_value=np.nan))
    landmarks = (landmarks.sort_index(level=[1, 0])
                 .assign(x=lambda x: x.groupby('landmark_index').x.apply(lambda g: g.interpolate()),
                         y=lambda x: x.groupby('landmark_index').y.apply(lambda g: g.interpolate()),
                         z=lambda x: x.groupby('landmark_index').z.apply(lambda g: g.interpolate()),
                  )
                 .sort_index(level=[0, 1])
                 .reset_index()
                )
    return landmarks

# 3D Plotly Viz

In [50]:
class Visualizer:
    COORD_COLS = ['x', 'y', 'z']
    
    def __init__(self, train):
        self.train = train
    
    def sample_by_sign(self, sign, random_state=69, mask=None):
        sign_mask = self.train.loc[:, 'sign'] == sign
        mask = sign_mask if mask is None else (sign_mask & mask)
        sample = self.train.loc[mask, :].sample(n=1, random_state=random_state)

        landmarks = pd.read_parquet(path / sample['path'].values[0])
        landmarks = landmarks.loc[~landmarks.loc[:, self.COORD_COLS].isna().all(axis=1), :]
        return sample, landmarks
    
    def visualize(self, sign, poses, random_state=69):
        self.sample, self.landmarks = self.sample_by_sign(sign, random_state)
        self.frames = self.landmarks.frame.unique().tolist()
        
        self.fig = make_subplots(rows=1, cols=len(poses),     
                                 specs=[[{"type": "scatter3d"} for i in range(len(poses))]],
                                )

        for i, pose in enumerate(poses):
            self.add_trace(pose, i+1)
        
        steps = list()
        for i, frame in enumerate(self.frames):
            step = dict(
                method='update',
                args=[{"visible": [False] * len(self.frames) * 6},
                      {"title": "Slider switched to step: " + str(frame)}],
                label=str(frame),
                value=i
            )
            step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
            for j in range(6):
                step["args"][0]["visible"][i + len(self.frames) * j] = True  # Toggle i'th trace to "visible"
            steps.append(step)

        sliders = [dict(
            active=1,
            currentvalue={'prefix': "Frame: "},
            pad={'t': 50},
            steps=steps
        )]
        
        self.fig.update_layout(
            sliders=sliders,
        )

        self.fig.show()
        
    def add_trace(self, landmarks_type, col):
        type_mask = self.landmarks.type == landmarks_type
        type_landmarks = self.landmarks.loc[type_mask, :]
        
        # markers
        for frame in self.frames:
            frame_landmarks = type_landmarks.loc[type_landmarks.frame == frame, ]
            self.fig.add_trace(
                go.Scatter3d(
                    visible=False, 
                    x=frame_landmarks.x, y=frame_landmarks.y, z=frame_landmarks.z, text=frame_landmarks.landmark_index,
                    marker=dict(size=8, opacity=0.8),
                    mode="markers",
                ),
                row=1, col=col,
            )

        # segments
        for frame in self.frames:
            frame_landmarks = type_landmarks.loc[type_landmarks.frame == frame, ]
            landmark_lines = create_landmark_lines(frame_landmarks, landmarks_type)
            self.fig.add_trace(
                go.Scatter3d(
                    visible=False, 
                    x=landmark_lines.x, y=landmark_lines.y, z=landmark_lines.z,
                    mode="lines",
                    hoverinfo='skip',
                ),
                row=1, col=col,
            )

def create_landmark_lines(frame_landmarks, landmarks_type='hand'):
    empty_row = pd.Series({
        'frame': None, 'row_id': None, 'type': None, 'x': None, 'y': None, 'z': None
    })
    frame_landmarks = frame_landmarks.set_index('landmark_index')
    
    if frame_landmarks.empty:
        line_indices = [None]
    elif landmarks_type in ['hand', 'left_hand', 'right_hand']:
        line_indices = [0, 1, None, 0, 5, None, 0, 17, None, 
                        1, 2, 3, 4, None,
                        5, 6, 7, 8, None,
                        5, 9, None,
                        9, 10, 11, 12, None,
                        9, 13, None,
                        13, 14, 15, 16, None,
                        13, 17, None,
                        17, 18, 19, 20, None
                       ]
    elif landmarks_type == "pose":
        line_indices = [0, 1, 2, 3, 7, None, 
                        0, 4, 5, 6, 7, 8, None,
                        11, 13, 15, 17, 19, 15, 21, None,
                        11, 12, 14, 16, 18, 20, 16, 22, None,
                        11, 23, 25, 27, 31, 29, 27, None,
                        12, 24, 26, 28, 30, 32, 28, None,
                        23, 24, None
               ]
    else:
        raise ValueError()
    lines = pd.DataFrame([frame_landmarks.loc[i, ] if i is not None else empty_row
                          for i in line_indices
                         ])
    return lines

In [51]:
viz = Visualizer(train)
viz.visualize('because', ['left_hand', 'pose', 'right_hand'], 72)